In [1]:
from mpi4py import MPI

import basix.ufl
import dolfinx
import numpy as np
import ufl

from fenicsx_ii import Average, Circle, LinearProblem, assemble_scalar

M = 32  # Number of elements in each spatial direction in the box
N = M  # Number of elements in the line
comm = MPI.COMM_WORLD
omega = dolfinx.mesh.create_box(
    comm,
    [[-0.5, -0.5, -0.5], [0.5, 0.5, 0.5]],
    [M, M, M],
    cell_type=dolfinx.mesh.CellType.tetrahedron,
)

In [3]:
l_min = -0.5
l_max = 0.5
if comm.rank == 0:
    nodes = np.zeros((N, 3), dtype=np.float64)
    nodes[:, 0] = np.full(nodes.shape[0], 0)
    nodes[:, 1] = np.full(nodes.shape[0], 0)
    nodes[:, 2] = np.linspace(l_min, l_max, nodes.shape[0])
    connectivity = np.repeat(np.arange(nodes.shape[0]), 2)[1:-1].reshape(
        nodes.shape[0] - 1, 2
    )
else:
    nodes = np.zeros((0, 3), dtype=np.float64)
    connectivity = np.zeros((0, 2), dtype=np.int64)
c_el = ufl.Mesh(
    basix.ufl.element("Lagrange", basix.CellType.interval, 1, shape=(nodes.shape[1],))
)
lmbda = dolfinx.mesh.create_mesh(
    comm,
    x=nodes,
    cells=connectivity,
    e=c_el,
    partitioner=dolfinx.mesh.create_cell_partitioner(
        dolfinx.mesh.GhostMode.shared_facet
    ),
)

In [4]:
degree = 1
V = dolfinx.fem.functionspace(omega, ("Lagrange", degree))
Q = dolfinx.fem.functionspace(lmbda, ("Lagrange", degree))

In [5]:
W = ufl.MixedFunctionSpace(*[V, Q])
(u, p) = ufl.TrialFunctions(W)
(v, q) = ufl.TestFunctions(W)

In [6]:
R = 0.05
q_degree = 20
restriction_trial = Circle(lmbda, R, degree=q_degree)
restriction_test = Circle(lmbda, R, degree=q_degree)

In [7]:
dx_3D = ufl.Measure("dx", domain=omega)
dx_1D = ufl.Measure("dx", domain=lmbda)

In [8]:
q_el = basix.ufl.quadrature_element(lmbda.basix_cell(), value_shape=(), degree=q_degree)
Rs = dolfinx.fem.functionspace(lmbda, q_el)

In [9]:
avg_u = Average(u, restriction_trial, Rs)
avg_v = Average(v, restriction_test, Rs)

In [10]:
alpha1 = dolfinx.fem.Constant(omega, 1.0)
A = ufl.pi * R**2
P = 2 * ufl.pi * R

In [11]:
xi = dolfinx.fem.Constant(omega, 1.0)
x = ufl.SpatialCoordinate(omega)

In [12]:
a = alpha1 * ufl.inner(ufl.grad(u), ufl.grad(v)) * dx_3D
a += P * xi * ufl.inner(avg_u - p, avg_v) * dx_1D
a += A * ufl.inner(ufl.grad(p), ufl.grad(q)) * dx_1D
a += P * xi * ufl.inner(p - avg_u, q) * dx_1D

In [13]:
p_ex = ufl.sin(ufl.pi * x[2]) + 2
xi_rate = xi / (xi + 1)

u_inside = xi_rate * p_ex
r = ufl.sqrt(x[0] ** 2 + x[1] ** 2)
u_outside = xi_rate * (1 - R * ufl.ln(r / R)) * p_ex
u_ex = ufl.conditional(r < R, u_inside, u_outside)

f_vol = -ufl.div(alpha1 * ufl.grad(u_ex))
A_fhat = -ufl.div(A * ufl.grad(p_ex)) + P * xi_rate * p_ex

L = f_vol * v * dx_3D
L += A_fhat * q * dx_1D

In [14]:
omega.topology.create_connectivity(omega.topology.dim - 1, omega.topology.dim)
exterior_facets = dolfinx.mesh.exterior_facet_indices(omega.topology)
exterior_dofs = dolfinx.fem.locate_dofs_topological(
    V, omega.topology.dim - 1, exterior_facets
)
bc_expr = dolfinx.fem.Expression(u_ex, V.element.interpolation_points)
u_bc = dolfinx.fem.Function(V)
u_bc.interpolate(bc_expr)
bc = dolfinx.fem.dirichletbc(u_bc, exterior_dofs)
bcs = [bc]

In [15]:
petsc_options = {
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
    "ksp_error_if_not_converged": True,
}
problem = LinearProblem(
    a,
    L,
    petsc_options_prefix="coupled_poisson",
    petsc_options=petsc_options,
    bcs=bcs,
)
uh, ph = problem.solve()
uh.name = "uh"
ph.name = "ph"

In [17]:
a

Form([Integral(Product(Product(FloatValue(0.3141592653589793), Constant(Mesh(blocked element (Basix element (P, tetrahedron, 1, gll_warped, unset, False, float64, []), (3,)), 0), (), 1)), Product(Sum(Product(IntValue(-1), Argument(FunctionSpace(Mesh(blocked element (Basix element (P, interval, 1, gll_warped, unset, False, float64, []), (3,)), 2), Basix element (P, interval, 1, gll_warped, unset, False, float64, [])), 1, 1)), Average(v_1^0, Circle(<dolfinx.mesh.Mesh object at 0x7c6dd437afd0>, radius=<function Circle.__init__.<locals>.<lambda> at 0x7c6dd43f2ca0>, num_points=11), FunctionSpace(<Mesh #2>, QuadratureElement(interval, array([[0.01088567],       [0.0564687 ],       [0.134924  ],       [0.24045194],       [0.36522842],       [0.5       ],       [0.63477158],       [0.75954806],       [0.865076  ],       [0.9435313 ],       [0.98911433]]), array([0.02783428, 0.06279018, 0.09314511, 0.11659688, 0.13140227,       0.13646254, 0.13140227, 0.11659688, 0.09314511, 0.06279018,       0

In [16]:
with dolfinx.io.VTXWriter(omega.comm, "u_3D_solver.bp", [uh]) as vtx:
    vtx.write(0.0)
with dolfinx.io.VTXWriter(lmbda.comm, "p_1D_solver.bp", [ph]) as vtx:
    vtx.write(0.0)